In [64]:
import os
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import evaluate
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from dataclasses import dataclass

@dataclass
class Config:
    RANDOM_STATE_SEED = 42
    MODEL_LIST = [
        # "japanese-asr/distil-whisper-large-v3-ja-reazonspeech-all",
        # "openai/whisper-tiny",
        # "openai/whisper-small",
        # "openai/whisper-medium",
        # "openai/whisper-large-v3",
        "./whisper-ja-asmr-tiny-1-earlyst/final"
    ]

In [65]:
df = pd.read_csv("./manifest/preprocessed-segments-index.csv")
df = df[df.lang == "ja"].reset_index(drop=True)
_, test_df = np.split(df.sample(frac=1, random_state=Config.RANDOM_STATE_SEED),
                      [int(len(df)*0.9)])
print(f"Loaded {len(test_df)} test samples")

Loaded 1698 test samples


/home/rehab/research/whisperlaz/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [66]:
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [67]:
def evaluate_model(model_name: str):
    processor = WhisperProcessor.from_pretrained(model_name)
    model     = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens      = []

    preds, refs = [], []
    for _, row in tqdm(test_df.iterrows(),
                       total=len(test_df),
                       desc=f"Eval {model_name}"):
        data = np.load(row["npz_path"], allow_pickle=True)
        audio_array = data["audio"].astype(np.float32)  # raw waveform
        transcript  = str(data["text"])

        # *** CORRECTED HERE ***
        inputs = processor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features.to(device)

        with torch.no_grad():
            generated_ids = model.generate(
                inputs,
                max_length=256,
                num_beams=1,
            )
        pred = processor.batch_decode(
            generated_ids, skip_special_tokens=True
        )[0]

        preds.append(pred)
        refs.append(transcript)

    wer = wer_metric.compute(predictions=preds, references=refs)
    cer = cer_metric.compute(predictions=preds, references=refs)
    return wer, cer

In [68]:
results = {}
for m in tqdm(Config.MODEL_LIST):
    wer, cer = evaluate_model(m)
    results[m] = {"wer": wer, "cer": cer}

  0%|          | 0/1 [00:00<?, ?it/s]

Eval ./whisper-ja-asmr-tiny-1-earlyst/final:   0%|          | 0/1698 [00:00<?, ?it/s]

ValueError: You have explicitly specified `forced_decoder_ids`. Please remove the `forced_decoder_ids` argument in favour of `input_ids` or `decoder_input_ids` respectively.

In [6]:
df_results = pd.DataFrame([
    {"model": model_name, "wer": metrics["wer"], "cer": metrics["cer"]}
    for model_name, metrics in results.items()
])
df_results

,model,wer,cer
0,japanese-asr/distil-whisper-large-v3-ja-reazon...,0.905581,0.283509
1,openai/whisper-tiny,1.581200,1.127423
2,openai/whisper-small,1.350399,0.528739
3,openai/whisper-medium,1.198070,0.560072
4,openai/whisper-large-v3,0.966429,0.310716


In [7]:
df_results.to_csv("./manifest/model-list-evals.csv")